<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_Deep_Learning_Con_TernsorFlow_Machine-Learning_e_IA-/blob/master/scripts/tema09/05-deepdream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#pip install tensorflow==1.14

# Deep Dream

In [0]:
!mkdir models/inception

In [8]:
!wget models/inception https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip

--2020-06-13 02:07:37--  http://models/inception
Resolving models (models)... failed: Name or service not known.
wget: unable to resolve host address ‘models’
--2020-06-13 02:07:37--  https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.13.128, 2a00:1450:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.13.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49937555 (48M) [application/zip]
Saving to: ‘inception5h.zip’

inception5h.zip     100%[===================>]  47.62M  37.3MB/s    in 1.3s    

2020-06-13 02:07:39 (37.3 MB/s) - ‘inception5h.zip’ saved [49937555/49937555]

FINISHED --2020-06-13 02:07:39--
Total wall clock time: 2.1s
Downloaded: 1 files, 48M in 1.3s (37.3 MB/s)


In [12]:
!unzip inception5h.zip

Archive:  inception5h.zip
  inflating: imagenet_comp_graph_label_strings.txt  
  inflating: tensorflow_inception_graph.pb  
  inflating: LICENSE                 


In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import tensorflow as tf
from io import BytesIO

In [0]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [0]:
model_nn = "models/inception5h/tensorflow_inception_graph.pb"
with tf.gfile.FastGFile(model_nn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    # vamos a leer de ese fichero .pb toda la info relativa de los nodos

In [0]:
t_input = tf.placeholder(np.float32, name='input')
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean,0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})

In [0]:
layers = [op.name for op in graph.get_operations() if op.type=="Conv2D" and "import/" in op.name]

In [23]:
layers

['import/conv2d0_pre_relu/conv',
 'import/conv2d1_pre_relu/conv',
 'import/conv2d2_pre_relu/conv',
 'import/mixed3a_1x1_pre_relu/conv',
 'import/mixed3a_3x3_bottleneck_pre_relu/conv',
 'import/mixed3a_3x3_pre_relu/conv',
 'import/mixed3a_5x5_bottleneck_pre_relu/conv',
 'import/mixed3a_5x5_pre_relu/conv',
 'import/mixed3a_pool_reduce_pre_relu/conv',
 'import/mixed3b_1x1_pre_relu/conv',
 'import/mixed3b_3x3_bottleneck_pre_relu/conv',
 'import/mixed3b_3x3_pre_relu/conv',
 'import/mixed3b_5x5_bottleneck_pre_relu/conv',
 'import/mixed3b_5x5_pre_relu/conv',
 'import/mixed3b_pool_reduce_pre_relu/conv',
 'import/mixed4a_1x1_pre_relu/conv',
 'import/mixed4a_3x3_bottleneck_pre_relu/conv',
 'import/mixed4a_3x3_pre_relu/conv',
 'import/mixed4a_5x5_bottleneck_pre_relu/conv',
 'import/mixed4a_5x5_pre_relu/conv',
 'import/mixed4a_pool_reduce_pre_relu/conv',
 'import/mixed4b_1x1_pre_relu/conv',
 'import/mixed4b_3x3_bottleneck_pre_relu/conv',
 'import/mixed4b_3x3_pre_relu/conv',
 'import/mixed4b_5x5_bo

## ACTIVAR GPU DE COLAB

In [50]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [51]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5324713073385642325, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7633786780155657016
 physical_device_desc: "device: XLA_CPU device"]

In [33]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12639515992280515938, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10778483703771150237
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
feature_num = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

In [53]:
feature_num

[64,
 64,
 192,
 64,
 96,
 128,
 16,
 32,
 32,
 128,
 128,
 192,
 32,
 96,
 64,
 192,
 96,
 204,
 16,
 48,
 64,
 160,
 112,
 224,
 24,
 64,
 64,
 128,
 128,
 256,
 24,
 64,
 64,
 112,
 144,
 288,
 32,
 64,
 64,
 256,
 160,
 320,
 32,
 128,
 128,
 256,
 160,
 320,
 48,
 128,
 128,
 384,
 192,
 384,
 48,
 128,
 128,
 128,
 128]

In [0]:
layer = "mixed4d_3x3_bottleneck_pre_relu"
channel = 139
im_noise = np.random.uniform(size = (224,224,3))+100.0

In [0]:
def show_array(a, fmt = "jpeg"):
    a = np.uint8(np.clip(a,0,1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    plt.imshow(a)

In [0]:
def T(layer):
    return graph.get_tensor_by_name("import/%s:0"%layer)

In [0]:
def tf_func(*argtypes):
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

In [0]:
def resize(img, size):
    img = tf.expand_dims(img,0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]

In [0]:
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h,w = img.shape[:2]
    sx, sy = np.random.randint(sz,size = 2)
    img_shift = np.roll(np.roll(img,sx,1),sy,0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz), sz):
        for x in range(0, max(w-sz//2, sz), sz):
            sub = img_shift[y:y+sz, x:x+sz]
            g = session.run(t_grad, {t_input:sub})
            grad[y:y+sz, x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy,0)

In [0]:
def render_deepdream(t_obj, img0=im_noise, iter_n = 10, step = 1.5, octave_n = 4, octave_scale = 1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]
    img = img0
    octaves = []
    for i in range(octave_n-1):
        hw = img.shape[:2]
        lo = resize(img, np.int32(np.float(hw)/octave_scale))
        hi = img-resize(lo, hw)
        img = lo
        octaves.append(hi)
    
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]#empezamos desde el último octave la reconstrucción
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g*(step / (np.abs(g).mean()+1e-7))
            print(".", end=" ")
        show_array(img/255.0)

In [0]:
resize = tf_func(np.float32, np.int32)(resize)